# Tutorial 2a. Tensors

- Tensors are one of the main ingredients when it comes to modern Deep-Learning frameworks. 
- Almost all deep learning computations can be expressed as tensor operations which make computation fast and efficient. 
- In this practical we will see how to manipulate tensors within the `PyTorch` library

You'll need to install `pytorch` using `conda install torch` or `pip install torch` on your laptop

In [ ]:
import torch
import numpy as np

## Basics with tensors

In [ ]:
# Generate a tensor of size 2x3x4
t = torch.Tensor(2, 3, 4)
print(t)

In [ ]:
# Get the type of the tensor
t.type()

In [ ]:
# Get information about the tensor
print(t.size())
print(t.shape)
print(t.numel())
print(t.dim())

In [ ]:
# Resize the tensor
r = torch.Tensor(t)
r.resize_(3, 8)
print(r)
print(r.size())

## Vectors (1D Tensors) and similarities with Numpy

- You might have noticed some similarities between the way PyTorch deals with tensors and the way Numpy deals with arrays
- When it comes to basic algebraic operations both libraries can indeed be very similar as shown in some of the examples below

In [ ]:
# Creates a 1D tensor of integers 1 to 4
v = torch.Tensor([1, 2, 3, 4])
np_v = np.array([1, 2, 3, 4])

print(v)
print(np_v)

Note however the difference with the default `dtypes` used

In [ ]:
v.dtype, np_v.dtype

In [ ]:
# We create a second tensor with the same properties
w = torch.Tensor([1, 0, 2, 0])
np_w = np.array([1, 0, 2, 0])

print(w)
print(np_w)

In [ ]:
# Element-wise multiplication
x = v * w
np_x = np_v * np_w

print(x)
print(np_x)

In [ ]:
# Slicing: extract sub-Tensor [from:to)
print(x[0:3])
print(np_x[0:3])

In [ ]:
# Create a tensor with integers ranging from 1 to 5, excluding 5
v = torch.arange(1, 4 + 1)
np_v = np.arange(1, 5)
print(v)
print(np_v)

In [ ]:
# Square all elements in the tensor
print(v ** 2)
print(np_v ** 2)

But also

In [ ]:
print(v.pow(2))
print(np.power(np_v, 2))

## Let's add one dimension: matrices (2D Tensors)

In [ ]:
# Create a 2x4 tensor
m = torch.Tensor([[2, 5, 3, 7],
                  [4, 2, 1, 9]])
print(m)

np_m = np.array([[2, 5, 3, 7],[4, 2, 1, 9]])
print(np_m)

In [ ]:
# Get the shape of the tensor
print(m.size())
print(np_m.shape)

In [ ]:
# Indexing row 0, column 2 (0-indexed)
m[0, 2]

In [ ]:
# Indexing column 1, all rows (returns size 2)

m[:, 1]

## Some easy exercises: tensor Operations

It is now up to you to get familiar with some basic tensor operations in PyTorch.

In [ ]:
m.dtype

In [ ]:
torch.Tensor()

In [ ]:
# Indexing column 1, all rows (returns size 2x1, not 2)

m[:, [1]]

In [ ]:
# Indexes row 0, all columns (returns 1x4)

m[[0], :]

In [ ]:
# Add a random tensor of size 2x4 to m

m + torch.rand(2, 4)

In [ ]:
# Subtract a random tensor of size 2x4 to m

m - torch.rand(2, 4)

In [ ]:
# Multiply a random tensor of size 2x4 to m

m * torch.rand(2, 4)

In [ ]:
# Divide m by a random tensor of size 2x4

m / torch.rand(2, 4)

In [ ]:
# Transpose tensor m

m.t()

## Exercise: polynomial regression

- Last week we discussed polynomial logistic regression
- Let's work also on polynomial regression (continuous label) with a **single feature**
- In this exercise we show how this simple model can be trained using `numpy` and your work will be to code the solution with `PyTorch` 

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
def true_f(x):
    return 0.1 * (x-2) ** 3 + x ** 2 - 8.0 * x - 1.0

def generate(n_samples):
    X = np.random.rand(n_samples) * 20.0 - 10.0
    y = true_f(X) + 5 * np.random.randn(n_samples)
    
    return X.reshape(n_samples, 1), y

X_train, y_train = generate(15)
xs = np.linspace(-10, 10, num=1000)
plt.plot(xs, true_f(xs), c="r", label="$g(x)$", lw=3)
plt.scatter(X_train, y_train, label="$y = g(x) + \epsilon$")
plt.legend(loc='upper left', fontsize=13)
plt.grid(color="grey")

In [ ]:
def numpy_model(X, beta, poly):
    Xp = poly.transform(X)
    return np.dot(Xp, beta)

In [ ]:
poly = PolynomialFeatures(degree=3, include_bias=True)
Xp = poly.fit_transform(X_train)

# We use the analytic solution for the optimization problem. 
# Note that this is not something we usually do in machine learning, where 
#   we'd rather use an optimization algorithm
np_beta = np.linalg.solve(Xp.T.dot(Xp), Xp.T.dot(y_train))
 # Least-squares error
error = np.mean((y_train - numpy_model(X_train, np_beta, poly)) ** 2)
plt.plot(xs, true_f(xs), c="r", label="$g(x)$")
plt.scatter(X_train, y_train, label="$y = g(x) + \epsilon$")
plt.plot(xs, numpy_model(xs.reshape(-1,1), np_beta, poly), c="b", label="$\hat{y} = f_{numpy}(x)$")
plt.title("degree = %d, $\hat{R}(f, d) = %.2f$" % (3, error), fontsize=15)
plt.ylim(-40, 80)
plt.grid()
plt.legend(fontsize=13)
plt.show()

print("The error of our model is {}".format(error))

# PyTorch solution

The solutons consists of 2 easy steps:

* create a function similar to the model one which does the required computations in PyTorch instead of Numpy.
* Solve the optimization problem by using the 2D tensor operations we have seen before. Some of these operations require some basic transformations between Numpy arrays and PyTorch tensors.

In [ ]:
def pytorch_model(X, beta, poly):
    Xp = poly.transform(X)    
    return torch.matmul(torch.from_numpy(Xp), beta)

In [ ]:
poly = PolynomialFeatures(degree=3, include_bias=True)

Xp = poly.fit_transform(X_train)
Xpt = torch.from_numpy(Xp)

In [ ]:
XtX = torch.matmul(Xpt.t(), Xpt)
Xty = torch.matmul(Xpt.t(), torch.from_numpy(y_train))
Xty = Xty.reshape(*Xty.shape, 1)
pt_beta, _ = torch.solve(Xty, XtX)

In [ ]:
pt_beta

In [ ]:
pt_beta - np_beta

Of course, we get the exact same solutions

In [ ]:
pytorch_model(X_train, pt_beta, poly)